In [ ]:
from pyincore import IncoreClient, FragilityService, MappingSet
from pyincore.analyses.waterfacilitydamage import WaterFacilityDamage
from pyincore.analyses.waterfacilityrepaircost import WaterFacilityRepairCost
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability

In [ ]:
client = IncoreClient()

In [ ]:
wf_id = "60e5e91960b3f41243faa3b2"

### Water Facility Damage

In [ ]:
# Facility damage    
# Initializing water facility damage and fragility service
wterfclty_dmg = WaterFacilityDamage(client)
fragility_service = FragilityService(client)

# loading input dataset
wterfclty_dmg.load_remote_input_dataset("water_facilities", wf_id)

# wterfclty Fragility Mapping on incore-service
mapping_id = "5d39e010b9219cc18bd0b0b6" # 5 DS
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))
wterfclty_dmg.set_input_dataset('dfr3_mapping_set', mapping_set)

wterfclty_dmg.set_parameter("hazard_type", 'earthquake')
wterfclty_dmg.set_parameter("fragility_key",'pga')
wterfclty_dmg.set_parameter("num_cpu", 4)
wterfclty_dmg.set_parameter("hazard_id", "5ba8f2b9ec2309043520906e")
wterfclty_dmg.set_parameter("result_name","seaside_wf_dmg_result")

# Run pipeline damage analysis
wterfclty_dmg.run_analysis()

In [ ]:
wf_dmg_result = wterfclty_dmg.get_output_dataset("result")
wf_dmg_result.get_dataframe_from_csv().head()

### Monte Carlo Simulation

In [ ]:
mc = MonteCarloFailureProbability(client)
mc.set_input_dataset("damage", wf_dmg_result)
mc.set_parameter("num_cpu", 4)
mc.set_parameter("num_samples", 100)
mc.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3", "DS_4"])
mc.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3", "DS_4"])

mc.set_parameter("result_name", "seaside_wf_mc") # name of csv file with results
mc.run()

In [ ]:
wf_sample_damage_states_result = mc.get_output_dataset("sample_damage_states")
wf_sample_damage_states_result.get_dataframe_from_csv().head()

### Water Facility Repair Cost

In [ ]:
wf_repair_cost = WaterFacilityRepairCost(client)

wf_repair_cost.load_remote_input_dataset("water_facilities", wf_id)
wf_repair_cost.load_remote_input_dataset("replacement_cost", "64833bcdd3f39a26a0c8b147")

# can be chained with MCS
wf_repair_cost.set_input_dataset("sample_damage_states", wf_sample_damage_states_result)

# dmg ratiose
wf_repair_cost.load_remote_input_dataset("wf_dmg_ratios", "647e423d7ae18139d9758607")

wf_repair_cost.set_parameter("result_name", "seaside_wf")
wf_repair_cost.set_parameter("num_cpu", 4)

# Run Analysis
wf_repair_cost.run_analysis()

In [ ]:
wf_repair_cost.get_output_dataset("result").get_dataframe_from_csv().head()